<a href="https://colab.research.google.com/github/PavelNovikov888/Anomaly_Detection/blob/master/%D0%94%D0%B5%D1%82%D0%B5%D0%BA%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B0%D0%BD%D0%BE%D0%BC%D0%B0%D0%BB%D0%B8%D0%B9_%D0%9A%D0%BB%D0%B0%D1%81%D1%82%D0%B5%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импортируем необходимые библиотеки

import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
#В колабе импортируем все необходимые библиотеки
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Привязываем гуглдиск к колабу
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Загружаем файл в колаб
download = drive.CreateFile({"id": "18TAIUdF23aChxJ36RH0t4DQNmvut9N7j"})
# https://drive.google.com/file/d/18TAIUdF23aChxJ36RH0t4DQNmvut9N7j/view?usp=sharing
download.GetContentFile("anomaly_sample.zip")

In [ ]:
# Загрузим данные

anomaly_df = pd.read_csv("anomaly_sample.zip")
anomaly_df.head()

,timestamp,value,is_anomaly,predicted
0,1425008573,42,False,44.072500
1,1425008873,41,False,50.709390
2,1425009173,41,False,81.405120
3,1425009473,61,False,39.950367
4,1425009773,44,False,35.350160


In [ ]:
# Посмотрим на данные
anomaly_df.describe()

,timestamp,value,predicted
count,1.583000e+04,15830.000000,15830.000000
mean,1.427383e+09,85.572205,71.870715
std,1.370962e+06,321.760918,92.450520
min,1.425009e+09,0.000000,-281.389070
25%,1.426196e+09,29.000000,32.919171
50%,1.427383e+09,47.000000,49.771124
75%,1.428570e+09,76.000000,75.948052
max,1.429757e+09,13479.000000,2716.127200


# Поищем аномалии, с помощью DBSCAN

In [ ]:
dbscan = DBSCAN(eps=1)

In [ ]:
anomaly_df['prediction_dbscan_1'] = dbscan.fit_predict(anomaly_df['value'].values.reshape(-1, 1))
anomaly_df.head()

,timestamp,value,is_anomaly,predicted,prediction_dbscan_1
0,1425008573,42,False,44.072500,0
1,1425008873,41,False,50.709390,0
2,1425009173,41,False,81.405120,0
3,1425009473,61,False,39.950367,0
4,1425009773,44,False,35.350160,0


In [ ]:
# Обнаруженные аномалии
anomaly_df[anomaly_df.prediction_dbscan_1 < 0]

,timestamp,value,is_anomaly,predicted,prediction_dbscan_1
162,1425057173,456,True,89.710290,-1
163,1425057473,440,True,134.684600,-1
164,1425057773,477,True,126.210050,-1
1006,1425310373,346,True,68.731980,-1
1168,1425358973,446,True,71.947266,-1
...,...,...,...,...,...
15464,1429647773,2510,True,407.389860,-1
15465,1429648073,1299,True,388.840450,-1
15466,1429648373,714,False,456.416630,-1
15467,1429648673,576,False,323.319300,-1


# Поиск аномалий с помощью агломеративной кластеризации

In [ ]:
agglom = AgglomerativeClustering(n_clusters=None, distance_threshold=1)

In [ ]:
anomaly_df['prediction_agglom_1'] = agglom.fit_predict(anomaly_df['value'].values.reshape(-1, 1))
anomaly_df.head()

,timestamp,value,is_anomaly,predicted,prediction_dbscan_1,prediction_agglom_1
0,1425008573,42,False,44.072500,0,2
1,1425008873,41,False,50.709390,0,0
2,1425009173,41,False,81.405120,0,0
3,1425009473,61,False,39.950367,0,15
4,1425009773,44,False,35.350160,0,1


In [ ]:
valcount = anomaly_df['prediction_agglom_1'].value_counts()
valcount = set(valcount[valcount == 1].index)
anomaly_df[anomaly_df.prediction_agglom_1.isin(valcount)]

,timestamp,value,is_anomaly,predicted,prediction_dbscan_1,prediction_agglom_1
162,1425057173,456,True,89.710290,-1,630
164,1425057773,477,True,126.210050,-1,629
1168,1425358973,446,True,71.947266,-1,628
1360,1425416573,1698,True,101.339670,-1,607
1361,1425416873,3228,True,148.821490,-1,479
...,...,...,...,...,...,...
15465,1429648073,1299,True,388.840450,-1,462
15466,1429648373,714,False,456.416630,-1,390
15467,1429648673,576,False,323.319300,-1,396
15468,1429648973,490,False,348.968020,16,577


Задание:
Сколько аномалий будет обнаружено при использовании DBSCAN с параметрами: число точек в кластере — 5, eps-окрестность — 2?

In [ ]:
dbscan_1 = DBSCAN(eps=2, min_samples=5)
anomaly_df['prediction_dbscan_1'] = dbscan_1.fit_predict(anomaly_df['value'].values.reshape(-1, 1))
# Обнаруженные аномалии
print(f'Количество аномалий: {anomaly_df[anomaly_df.prediction_dbscan_1 < 0].shape[0]}')

Количество аномалий: 236


Задание:
Сколько аномалий будет обнаружено при использовании DBSCAN с параметрами: число точек в кластере — 10, eps-окрестность — 5?

In [ ]:
dbscan_2 = DBSCAN(eps=5, min_samples=10)
anomaly_df['prediction_dbscan_2'] = dbscan_2.fit_predict(anomaly_df['value'].values.reshape(-1, 1))
# Обнаруженные аномалии
print(f'Количество аномалий: {anomaly_df[anomaly_df.prediction_dbscan_2 < 0].shape[0]}')

Количество аномалий: 226


Задание:
Сколько аномалий будет обнаружено при использовании агломеративной кластеризации с дистанцией отсечения 2?

In [ ]:
agglom_2 = AgglomerativeClustering(n_clusters = None, distance_threshold=2)
anomaly_df['prediction_agglom_2'] = agglom_2.fit_predict(anomaly_df['value'].values.reshape(-1, 1))
valcount = anomaly_df['prediction_agglom_2'].value_counts()
valcount = set(valcount[valcount == 1].index)
print(f'Количество аномалий: {anomaly_df[anomaly_df.prediction_agglom_2.isin(valcount)].shape[0]}')

Количество аномалий: 147
